## ARXModel Tests

This Notebooks is dedicated to test the output of python based invx and detection source code

In [17]:
%load_ext autoreload
%autoreload 2

import importlib
try:    importlib.reload(Jupytils)
except: import Jupytils
import logging as log
import itertools as it
showTopbar('''ARX Model Create''')
from invx import *

from sklearn.linear_model import LinearRegression
#from arch.univariate import ARX

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [18]:
# * NOTE * : Before you run this, make sure you have run the following code:
#   [ ] python invx.py "data/test.csv"
#   [ ] test.xml is NEC SIAT program created File

df=LoadDataSet("data/test.csv")
dfi=LoadDataSet("data/test.xml", xmlTag="Invariant")
dfi['theta1']= dfi.apply (lambda row: [ float(i.strip()) for i in  row['theta'].split(',') if i.strip()], axis=1)
dfi.n = dfi.n.astype(int)
dfi.m = dfi.m.astype(int)
dfi.k = dfi.k.astype(int)
dfi.fitness = dfi.fitness.astype(float)
dfi.threshold = dfi.threshold.astype(float)
dfi.sort_values(['uName', 'yName'], inplace=True)
dfi = dfi.reset_index(drop=True)
dfi = dfi["uName,yName,fitness,correlation,n,m,k,threshold,theta1".split(',')];
displayDFs([df,dfi], showIcons=False);


In [19]:
%time dfii = CreateInvariants("data/test.csv", "data/test.csv.model.csv")
displayDFs(dfii);

INFO:root:Creating invariants using Index(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='object')


CPU times: user 3.26 s, sys: 363 ms, total: 3.62 s
Wall time: 2.02 s


,uName,yName,fitness,correlation,n,m,k,threshold,theta
0,A,B,0.836,0.986,0,1,0,4.618,"0.7566798046979863,0.08742089615344534,76.55782422422541"
1,A,D,0.297,0.628,1,0,2,3.407,"0.963013342470833,0.02304386659411589,0.10041268310138918"
2,A,E,0.162,0.078,0,1,0,2.284,"0.3436714758845351,-0.34042215819692534,24.711512869990973"
3,A,F,0.721,0.942,1,1,2,5.690,"0.7120381334303055,0.20514306199551868,0.18059555568252303,-21.856425676201695"
4,A,G,0.125,-0.481,1,1,0,3.285,"0.534507356153688,-0.002926132271156909,-0.04409893143921596,27.323369170865533"
5,B,A,0.843,0.986,0,1,0,4.155,"0.5722391154920231,0.583804426656154,-86.36247904426479"


In [31]:
#%%time
TEST=0      # NOT an essential test
if (TEST):
    '''
    This will Test if find Best Function works as expected 
    '''
    nErrs = 0;
    for i in range(len(dfi)):
        loc=i
        u,v,f,c,n,m,k,thr,etheta=dfi.loc[loc]
        x=df[u].values    
        y=df[v].values
        best=findBest(y,x)
        ret = best
        inv1 = [u,v,ret.fitscore, 1, ret.res, ret.n, ret.m, ret.k, ret.threshold]
        if ( not np.allclose(f, best.fitscore) or (n,m,k) != best.nmk  or not np.allclose(thr, best.threshold)):
            print(u,v,ret.fitscore, ",".join([str(c) for c in ret.res]), ret.n, ret.m, ret.k, ret.threshold)
            print(u,v,f,n,m,k,thr,etheta)
            print(f'ERROR: MISMATCH {u,v} Fit: {f} === {best.fitscore}, \nnmk: {(n,m,k)} === {best.nmk}, \n Threshhold: {thr} === {best.threshold}')
            nErrs+=1
            if (nErrs > 0):
                break;
        else:
            pass;
        #break;
    if (nErrs <=0):
        print("* PASSED ")
else:
    print("PASSED: non-essential Test")

In [50]:
'''
This will test call to ARXModel
'''
def testARXModelLR(dfi, loc=0, log=True):
    u,v,f,c,n,m,k,thr,etheta=dfi.loc[loc]

    x=df[u].values    
    y=df[v].values
    x1=x.reshape((len(x),1))
    logd(log,f'{loc}: {u,v} n,m,k ={n,m,k} theta={ np.array(etheta)}' )
    #res, arx = ARXModelOld(y, x1, n, m, k);
    ret, arx = ARXModelLR(y, x1, n, m, k);
    ret[0:n] *= -1;  #We need this to make it match original file
    np.set_printoptions(precision=6)
    logd(log, ret)
    
    comp = np.allclose(ret,etheta)
    logd(log, "Matched: ", comp)
    if ( not comp):
        print (f"**MISMATCH :{loc} {n,m,k} {u, v } Not Matched\n {ret} \n {etheta}")
        return comp;
    return ret, comp, etheta;

TEST=0;
if ( TEST):
    for i in range(30):
        ret, comp, etheta = testARXModelLR(dfi, i, False)
    if ( not comp ):
        print("**FAILED **")
    else:
        print("**PASSED **")
else:
    print("PASSED: non-essential Test")

PASSED: non-essential Test


In [65]:
'''
This will test call to FitnessScore call
'''
def testFitnessScore(dfi, loc = 0, log=True):
#loc=9
    u,v,f,c,n,m,k,thr,etheta=dfi.loc[loc]

    x=df[u].values    
    y=df[v].values
    x1=x.reshape((len(x),1))
    #logd(log,f'{loc}: {u,v} n,m,k ={n,m,k} theta={ np.array(theta)}' )
    theta1, arx = ARXModelLR(y,x1,n,m,k)
    fitscore,yh,rs = FitnessScore(x,y,n,m,k, theta1, False)

    ret = np.allclose(fitscore, f)
    if (not ret):
        print(f"*MISMATCH => {loc} {n,m,k}  \n {f}: {etheta} \n {fitscore}: {theta1}")
        return (ret)
    return 0
TEST=0;
if ( TEST):
    for i in range(len(dfi)):
        ret = testFitnessScore(dfi, i, False)
        
    if ( ret ):
        print("**FAILED **")
    else:
        print("**PASSED **")
else:
    print("PASSED: non-essential Test")        
  

PASSED: non-essential Test


In [88]:
'''
This is non-essential - I will just leave it here to show usage of predicts
and comapre performance of jit.
'''

'''from invx import *

def cmppredicts():
    x=df['A'].values    
    y=df['B'].values
    yhat=y#np.array(y.copy())
    loc= 5
    u1,v1,f1,c1,_,t1,n1,m1,k1,_,_,_,theta1=dfi.loc[loc]

    print(u1, v1, theta1, n1,m1, k1)
    a1,b1 = predict1(x, y, n1,m1,k1, theta1, loc)
    a2,b2 = predict2(x, y, n1,m1,k1, theta1, loc)
    a3,b3 = predict3(x, y, n1,m1,k1, theta1, loc)

    print(a1, b1, "==", a2, b2, "==", a3, b3)

def predict30(x, y, n,m,k, theta, t):
    yh = theta[-1];
    for i in range(n):
        yh += y[t-n+i] * theta[n-1-i]
    for i in range(m+1):
        yh += x[t-k-i] * theta[i+n]
    
    rs = (y[t] - yh)
    return yh ,rs

@jit(nopython=True, cache=True)
def predict31(x, y, n,m,k, theta, t):
    yh = theta[-1];
    for i in range(n):
        yh += y[t-n+i] * theta[n-1-i]
    for i in range(m+1):
        yh += x[t-k-i] * theta[i+n]
    
    rs = (y[t] - yh)
    return yh ,rs

def test(t=0,n=1000):
    x=df['A'].values    
    y=df['B'].values
    yhat=y#np.array(y.copy())
    loc= 5
    u1,v1,f1,c1,_,t1,n1,m1,k1,_,_,_,theta1=dfi.loc[loc]
    for i in range(n):
        if (t == 0 ):
            predict30(x,y, n1,m1,k1,theta1,loc)
        elif(t==1):
            predict31(x,y, n1,m1,k1,theta1,loc)
        else:
            predict32(x,y, n1,m1,k1,theta1,loc)'''
            
print ("*PASS NON ESSENTIAL")

*PASS NON ESSENTIAL


In [32]:
%%writefile  flatteninvxml.py
#!/usr/local/bin/python 
import Jupytils
import sys
from Jupytils.DataFrameUtils import *

def thetaF1(ret):
    t = np.array([ float(i.strip()) for i in  ret['theta'].split(',') if i.strip()])
    t[0:ret.n] *= -1;
    return t

def thetaF2(ret):
    t = [ float(i.strip()) for i in  ret['theta'].split(',') if i.strip()]
    return t

def thetaF1Str(ret):
    t = np.array([float(i.strip()) for i in  ret['theta'].split(',') if i.strip()])
    t[0:ret.n] *= -1;
    return str(list(t))[1:-1]

def LoadInvFile(file, needTheta1=True):
    if ( file.endswith("xml")):
        dfi1=LoadDataSet( file, xmlTag="Invariant")
        dfi1 = dfi1["uName,yName,fitness,correlation,n,m,k,threshold,theta".split(',')];
    else:
        dfi1=pd.read_csv(file)
        
    dfi1.uName = dfi1.uName.str.strip()
    dfi1.yName = dfi1.yName.str.strip()
    dfi1.n = dfi1.n.astype(int)
    dfi1.m = dfi1.m.astype(int)
    dfi1.k = dfi1.k.astype(int)
    dfi1.fitness = dfi1.fitness.astype(float)
    dfi1.threshold = dfi1.threshold.astype(float)     
   
    if ( needTheta1):
        if ( file.endswith("xml")):
            dfi1['theta1']= dfi1.apply (lambda row: thetaF1(row),axis=1)  # Convert to float array
        else:
            dfi1['theta1']= dfi1.apply (lambda row: thetaF2(row),axis=1)  # Convert to float array

    dfi1.sort_values(['uName', 'yName'], inplace=True)
    dfi1.reset_index(inplace=True, drop= True)
    return dfi1

def inJupyter():
    try:get_ipython; return True
    except: return False;


if __name__ == '__main__':
    if (not inJupyter()):
        if (len(sys.argv) < 2 ): 
            print("flatteninv.xml <inv.xml file>")
            sys.exit(1)
        print("processing ", sys.argv[1])
        df=LoadInvFile(sys.argv[1], needTheta1=False);
        df['theta'] = df.apply (lambda row: thetaF1Str(row),axis=1)
        df.to_csv( sys.argv[1]+".csv", index=False)


Overwriting flatteninvxml.py


In [414]:
#%%writefile  cmp.py
#!/usr/local/bin/python 
from invx import *
import logging as log
import sys
import os
import pandas as pd
import numpy as np;
import datetime
import getopt
from collections import defaultdict
import math
import re;
import gc
import json
from Jupytils import Map
import Jupytils
from Jupytils.DataFrameUtils import *

'''
This will compare their generated XML FIle to our generated XML FIle and ensure they match;
Very very important Test case
'''

def AreTwoRowsSame(dfi1, dfi2, loc =0):
#    u1,v1,f1,c1,_,t1,n1,m1,k1,_,_,_,theta1=dfi1.loc[loc]
    u1,v1,f1,c1,n1,m1,k1,th1,t1,theta1=dfi1.loc[loc]
    
    r = dfi2[(dfi2.uName == u1) & ( dfi2.yName == v1)]
    #u,v,f,c,_,t,n,m,k,thr,theta=dfi2.loc[loc]
    
    if ( len(r) < 1):
        #print(f'could not find {u1}-{v1} in dfi2')
        return True;
        
    u2,v2,f2,c2,n2,m2,k2,th2,t2,theta2=r.values[0]
    theta2 = np.array(theta2)
    
    ret = True
    if ( not np.allclose(f1, f2) ):
        print(f"*ERROR* Location: {loc} {u1}:{v1} => Fitness score not match {f1} != {f2}")
        ret = False
        
    if (n1 != n2 or m1 != m2 or k1 != k2):
        print(f"*ERROR* Location: {loc} {u1}:{v1} => nmk not match {n1, m1, k1} != {n2, m2, k2}")
        ret = False

    if ( len(theta1) == len(theta2) and not np.allclose(theta1, theta2, atol=1e-6, rtol=1e-04) ):
        print(f"*ERROR* Location: {loc} {u1}:{v1} => Parameters didnt match {theta1} != {theta2}")
        ret = False

    if ( not ret):
        pass;
        #print (len(r), u1, v1, r)
    else:
        pass; #print(loc, ' ', end='')    
    return ret;

def thetaF(ret, xml=True):
    t = np.array([ float(i.strip()) for i in  ret['theta'].split(',') if i.strip()])
    if (xml):
        t[0:ret.n] *= -1;
    return t

def LoadInvFile(file):
    if ( file.endswith("xml")):
        dfi1=LoadDataSet( file, xmlTag="Invariant")
        dfi1 = dfi1["uName,yName,fitness,correlation,n,m,k,threshold,theta".split(',')];
    else:
        dfi1=pd.read_csv(file)
        
    dfi1.uName = dfi1.uName.str.strip()
    dfi1.yName = dfi1.yName.str.strip()
    dfi1.n = dfi1.n.astype(int)
    dfi1.m = dfi1.m.astype(int)
    dfi1.k = dfi1.k.astype(int)
    dfi1.fitness = dfi1.fitness.astype(float)
    dfi1.threshold = dfi1.threshold.astype(float)     
   
    if ( file.endswith("xml")):
        dfi1['theta1']= dfi1.apply (lambda row: thetaF(row),axis=1)  # Convert to float array
    else:
        dfi1['theta1']= dfi1.apply (lambda row: thetaF(row,0),axis=1)  # Convert to float array

    dfi1.sort_values(['uName', 'yName'], inplace=True)
    dfi1.reset_index(inplace=True, drop= True)
    return dfi1
        
def TestCompareInvCSV(invFile, invCSV, log=True, breakOnMisMatches=3, start=0, end=10*1024*1024):
    dfi1 = LoadInvFile(invFile) if type(invFile) == str else invFile
    dfi2 = LoadInvFile(invCSV)  if type(invCSV)  == str else invCSV

    allUVs1   = np.unique( list(dfi1.uName.values) + list(dfi1.yName.values) )
    allUVs2   = np.unique( list(dfi2.uName.values) + list(dfi2.yName.values) )

    s1 = set(allUVs1)
    s2 = set(allUVs2)
    if (s1-s2):
        print(f"There are {len(s1-s2)} sensors in 1 and not in 2:",  list(s1-s2)[0:10] )
    if (s2-s1):
        print(f"There are {len(s2-s1)} sensors in 2 and not in 1:",  list(s2-s1)[0:10] )
    if( len(allUVs1) != len(allUVs2) ):
        print("Not all U, V's present {len(allUVs1)} {len(allUVs2)}")
        
    if( len(dfi1) != len(dfi2) ):
        print(f"*MISMACTH Files have different Length: :{len(dfi1)}, :{len(dfi2)}")
        #return dfi1, dfi2
    else:
        #print(f"* PASS: Lengths Match: {invFile}:{len(dfi1)}, {invCSV}:{len(dfi2)}")
        print(f"* PASS: Lengths Match: :{len(dfi1)}, :{len(dfi2)}")

    i, errs, ret = 0, 0, True;
    end1 = max(len(dfi1), len(dfi2))
    end1 = min(end1, end)
    
    for i in range(start, end):
        ret = AreTwoRowsSame(dfi1, dfi2, i)
        if ( not ret):
            errs += 1;
        if (errs >= breakOnMisMatches):
            break
            
    print(f'* {errs} ERRORS Compared {i+1} Rows')    
    return dfi1, dfi2

def Usage():
    print('''Usage: sys.argv[0]} file1 file2 
          Ex: sys.argv[0]}  test.inv.xml test.inv.csv
          ''')

def main():
    global GLOBAL_ARGS
    args = GLOBAL_ARGS['__ARGS__']
    if (len(args) < 2 ): 
        Usage();
        print(f"**ERROR: REQUIRED Parameters not given {len(args)} *\n\n");
        return;
    file1 = args[0]
    file2 = args[1]
    dfi1, dfi2 = TestCompareInvCSV (file1, file2)
    #cFrom = int(GLOBAL_ARGS['-f']) if ('-f' in GLOBAL_ARGS) else 0
    #cTo = int(GLOBAL_ARGS['-t']) if ('-t' in GLOBAL_ARGS) else 100000
    #CreateInvariants( csvp, outp, cFrom, cTo)
    
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        getargs("-hf:t:")
        main()
        t2 = datetime.datetime.now()
        print(f"All Done in {str(t2-t1)} ***")
    else:
        pass #dfi1, dfi2 = testCompareInvCSV ("/EXTDATA/app1/test/rtest.inv.xml",'/EXTDATA/app1/test/rtest.model')
        #dfi1, dfi2 = TestCompareInvCSV ("data/test.inv.xml", "data/test.csv.inv.csv")


In [412]:
invxml='/NEC/SIAT-OLD/SIAT-OLD/benchmarks/test1.inv.xml'
invcsv='/NEC/SIAT-OLD/SIAT-OLD/benchmarks/test1.csv.model.csv'
invxml='/NEC/SIAT-OLD/SIAT-OLD/benchmarks/normal1.inv.xml'
invcsv='/NEC/SIAT-OLD/SIAT-OLD/benchmarks/normal1.csv.model.csv'

#dfx=LoadDataSet(invxml, xmlTag="Invariant")
#dfv=pd.read_csv(invcsv)
#dfv
#dfi1 = LoadInvFile(invxml)
#dfi2 = LoadInvFile(invcsv)
#allUVs1   = np.unique( list(dfi1.uName.values) + list(dfi1.yName.values) )
#allUVs2   = np.unique( list(dfi2.uName.values) + list(dfi2.yName.values) )

s1 = set(allUVs1)
s2 = set(allUVs2)
if (s1-s2):
    print(f"There are {len(s1-s2)} sensors in 1 and not in 2:",  list(s1-s2)[0:10] )
if (s2-s1):
    print(f"There are {len(s2-s1)} sensors in 2 and not in 1:",  list(s2-s1)[0:10] )

if( len(allUVs1) != len(allUVs2) ):
    print("Not all U, V's present {len(allUVs1)} {len(allUVs2)}")

#dfi1, dfi2 = TestCompareInvCSV (invxml, invcsv, True, 50, 5600, 5620)
#dfi1, dfi2 = TestCompareInvCSV (dfi1, dfi2, True, 50, 26116, 27 *1024 )
#dfv=pd.read_csv(invcsv)
#dfv

There are 5 sensors in 1 and not in 2: ['QIC103.SV', 'PIC401.SV', 'FIC403.SV', 'FIC301.SV', 'FIC302.SV']
Not all U, V's present {len(allUVs1)} {len(allUVs2)}


In [419]:
%%writefile  combinescores.py
#!/usr/local/bin/python 
import pandas as pd
import numpy as np;
import datetime, sys
import xml.etree.ElementTree as ET
from operator import itemgetter
import json

from operator import itemgetter
def getScoreTable(sdf, idx=0, maxRows=1000):
    cols=sdf.columns[1:];
    row=sdf.ix[0][1:]
    ret = [[cols[i], row[i]] for i in range(1,len(cols)-1) if row[i] > 0 ]
    #for ri in ret:
    #    ri.append(len(ri)) 
    ret=sorted(ret, key=itemgetter(1), reverse=True)
    return ret[0:maxRows]

def LoadBrknXML(brkInvFile, xmlTag = "RCResult"):
    with open(brkInvFile, "r") as f:
        xmlText = f.read()

    root = ET.fromstring(xmlText)
    iters= root.findall(path=xmlTag) if (xmlTag) else root

    # This will write columns as follows
    # period, anomalyScore, brokenCount, [[uname, yname, residual, significance]], []
    ret = []
    for i,e in enumerate(iters):
        (p_, a_, b_) = int(e.findtext('period')), float(e.findtext('anomalyScore')), int(e.findtext('brokenCount'))
        bi = ""
        iters1= e.findall('brokenInvariant')
        ret1=[]
        for i1,e1 in enumerate(iters1):
            (u, y, res, sig) =  e1.findtext('uName'),     e1.findtext('yName'),   \
                                e1.findtext('residual'), e1.findtext('significance');
            ret1.append((u, y, res, sig))
            
        ret1=sorted(ret1,key=itemgetter(3))

        bi = json.dumps(ret1)
        ret.append((p_, 0, a_, b_, bi,0))
        
    return ret;

def main(argv):
    if (len(argv) <= 1):
        print ("*ERROR* need prefix - expecting files in format _BrokenInv.csv,_Score.csv")
        return 
    
    pref=argv[1] #"ab1.csv_siat"

    brkn=LoadBrknXML(f"/NEC/SIAT-OLD/SIAT-OLD/benchmarks/{pref}_BrokenInv.csv")
    scoreDF=pd.read_csv(f"/NEC/SIAT-OLD/SIAT-OLD/benchmarks/{pref}_Score.csv")

    brk=pd.DataFrame(brkn)
    brk.columns='period,time,score,brknCount,brknPairs,brknSensors'.split(',')
    brk.time=scoreDF.Time
    bs = [getScoreTable(scoreDF,i) for i in range(scoreDF.shape[0])]
    brk.brknSensors = bs

    brk.to_csv(f"/NEC/SIAT-OLD/SIAT-OLD/benchmarks/{pref}.out", index=False)

    
def inJupyter():
    try:get_ipython; return True
    except: return False;

if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        main(sys.argv)
        t2 = datetime.datetime.now()
        print(f"All Done in {str(t2-t1)} ***")
    else:
        pass #d

Overwriting combinescores.py
